In [1]:
from time import sleep, perf_counter
import concurrent.futures
from bs4 import BeautifulSoup 
import requests
import json
import gzip
import pandas as pd

In [2]:
host_cities = []
events_url = "https://www.olympedia.org/editions"
response = requests.get(events_url)
if response.status_code == 200:
    events_page = BeautifulSoup(response.content, "lxml")
    summer_table = events_page.find_all("table")[0]
    winter_table = events_page.find_all("table")[1]
    for row in summer_table.find_all("tr")[1:]:
        year = row.find_all('td')[1].text
        event = {
            "Year": year,
            "Season": "Summer",
            "Game": f"{year} Summer Olympics",
            "Host_City": row.find_all("td")[2].text
        }
        host_cities.append(event)
    for row in winter_table.find_all("tr")[1:]:
        year = row.find_all('td')[1].text
        event = {
            "Year": year,
            "Season": "Winter",
            "Game": f"{year} Winter Olympics",
            "Host_City": row.find_all("td")[2].text
        }
        host_cities.append(event)
else:
    print(response.raise_for_status())


In [3]:
host_cities_df = pd.DataFrame(host_cities)
host_cities_df.to_csv("data/host_cities.csv")
host_cities_df.head(5)

,Year,Season,Game,Host_City
0,1896,Summer,1896 Summer Olympics,Athina
1,1900,Summer,1900 Summer Olympics,Paris
2,1904,Summer,1904 Summer Olympics,St. Louis
3,1908,Summer,1908 Summer Olympics,London
4,1912,Summer,1912 Summer Olympics,Stockholm


In [4]:
with open('data/athletes_urls.json', 'r') as file:
    athletes_urls = json.load(file)
print(len(athletes_urls))
athlete_total_content = []

155943


In [5]:
def athletes_content(athlete_urls):
    response = requests.get(athlete_urls)
    if response.status_code == 200:
        athletes_page = BeautifulSoup(response.content, "lxml")
        name = athletes_page.find("h1")
        biography = athletes_page.find(attrs = {"class": "biodata"})
        game_result = athletes_page.find('table', attrs = {"class": "table"})
        content = "<div>" + str(name) + "\n" + f"<h4>{athlete_urls}</h4>" + str(biography) + "\n" + str(game_result) + "</div>"
        athlete_total_content.append(content)
    else:
        print(response.raise_for_status())

In [6]:
start = perf_counter()
with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
    executor.map(athletes_content, athletes_urls)
end = perf_counter()
print(f"Elapsed Time: {end-start:.3f} seconds")

In [7]:
with gzip.open("data/athletes_content.json.gz", 'wb') as file:
    file.write(json.dumps(athlete_total_content).encode('utf-8'))

In [2]:
with gzip.open("data/athletes_content.json.gz", 'rb') as file:
    decompressed_data = file.read().decode('utf-8')

total_athletes_content = json.loads(decompressed_data)

In [3]:
print(total_athletes_content[3])

<div><h1>
Ángel Sposato
</h1>
<h4>https://www.olympedia.org/athletes/55113</h4><table class="biodata">
<tr><th>Roles</th><td>Competed in Olympic Games</td></tr>
<tr><th>Sex</th><td>Male</td></tr>
<tr><th>Full name</th><td>Ángel Edmundo•Sposato</td></tr>
<tr><th>Used name</th><td>Ángel•Sposato</td></tr>
<tr><th>Born</th><td>17 May 1922</td></tr>
<tr><th>Measurements</th><td>74 kg</td></tr>
<tr><th>NOC</th><td><img src="https://olympedia-flags.s3.eu-central-1.amazonaws.com/ARG.png" style="padding-right: 2px; vertical-align: middle"/> <a href="/countries/ARG">Argentina</a></td></tr>
</table>
<table class="table">
<thead>
<tr>
<th>Games</th>
<th>Discipline (Sport) / Event</th>
<th>NOC / Team</th>
<th>Pos</th>
<th>Medal</th>
<th>As</th>
<th></th>
</tr>
</thead>
<tbody>
<tr class="active">
<td>
<a href="/editions/13">1952 Summer Olympics</a>
</td>
<td><a href="/sports/WLF">Weightlifting</a></td>
<td><a href="/countries/ARG"><img src="https://olympedia-flags.s3.eu-central-1.amazonaws.com/ARG.

In [6]:
def extract_data(content):
    athletes_statistics = []
    athlete_info = BeautifulSoup(content, 'lxml')

    id = int(athlete_info.find("h4").text.split("/")[-1])
    name = athlete_info.find("h1").text.strip()
    gender, born, died, weight, height, noc = None, None, None, None, None, None
    biography = athlete_info.find(attrs = {"class":"biodata"}).find_all('tr')
    for row in biography:
        header = row.find("th").text
        data = row.find("td").text
        if header == "Sex":
            gender = data
        elif header == "Born":
            born = data.split('in')[0].strip()
        elif header == 'Died':
            died = data.split('in')[0].strip()
        elif header == "Measurements":
            measurements = data.split(' / ')
            if len(measurements) == 2: 
                height, weight = measurements 
            elif "kg" in data:
                weight = data
            elif "cm" in data:
                height = data
        elif header == 'NOC':
            noc = data.strip()
    
    biography_games_table = athlete_info.find('tbody').find_all('tr')
    game, sport, team = None, None, None
    event, position, medal = None, None, None
    game_temp = None
    for row in biography_games_table:
        if row.has_attr('class'):
            game, sport, team = [td.text.strip() for td in row.find_all('td')[:3]]
            if game != "":
                game_temp = game
            else:
                game = game_temp
        else:
            event = f"{sport}, {row.find_all('td')[1].text.strip('n')}".replace('\n', ''),
            position = row.find_all('td')[3].text,
            medal = row.find_all('td')[4].text
            athlete_data = {
                'id' : id,
                'name' : name,
                'gender' : gender,
                'born' : born,
                'died' : died,
                'height' : height,
                'weight' : weight,
                'noc' : noc,
                'game' : game,
                'team' : team,
                'sport' : sport,
                'event' : event,
                'position': position,
                'medal' : medal
            }
            athletes_statistics.append(athlete_data)
    return(athletes_statistics)

In [7]:
start = perf_counter()
with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
    all_athletes_data = list(executor.map(extract_data, total_athletes_content))   
end = perf_counter()
print(f"Elapsed Time: {end-start:.02f} seconds")

Elapsed Time: 844.90 seconds


In [8]:
flatten_athletes_data = []
for athlete_stats in all_athletes_data:
    for athlete_data in athlete_stats:
        flatten_athletes_data.append(athlete_data)

In [9]:
print(flatten_athletes_data[:6])

[{'id': 111427, 'name': 'Cui Peng', 'gender': 'Male', 'born': '31 May 1987', 'died': None, 'height': '179 cm', 'weight': '72 kg', 'noc': "People's Republic of China", 'game': '2008 Summer Olympics', 'team': 'CHN', 'sport': 'Football (Football)', 'event': ('Football (Football), Football, Men(Olympic)',), 'position': ('13',), 'medal': ''}, {'id': 97418, 'name': 'Muzaffer Selçuk', 'gender': 'Male', 'born': '12 August 1961', 'died': None, 'height': '180 cm', 'weight': '74 kg', 'noc': 'Türkiye', 'game': '1984 Winter Olympics', 'team': 'TUR', 'sport': 'Cross Country Skiing (Skiing)', 'event': ('Cross Country Skiing (Skiing), 15 kilometres, Men(Olympic)',), 'position': ('DNF',), 'medal': ''}, {'id': 18579, 'name': 'Georges Breuille', 'gender': 'Male', 'born': None, 'died': None, 'height': None, 'weight': None, 'noc': 'France', 'game': '1920 Summer Olympics', 'team': 'FRA', 'sport': 'Hockey', 'event': ('Hockey, Hockey, Men(Olympic)',), 'position': ('4',), 'medal': ''}, {'id': 55113, 'name': 'Á

In [10]:
athletes_data_df = pd.DataFrame(flatten_athletes_data)

In [3]:
host_cities_df = pd.read_csv("data/host_cities.csv")
host_cities_df

,Unnamed: 0,Year,Season,Game,Host_City
0,0,1896,Summer,1896 Summer Olympics,Athina
1,1,1900,Summer,1900 Summer Olympics,Paris
2,2,1904,Summer,1904 Summer Olympics,St. Louis
3,3,1908,Summer,1908 Summer Olympics,London
4,4,1912,Summer,1912 Summer Olympics,Stockholm
...,...,...,...,...,...
57,57,2010,Winter,2010 Winter Olympics,Vancouver
58,58,2014,Winter,2014 Winter Olympics,Sochi
59,59,2018,Winter,2018 Winter Olympics,PyeongChang
60,60,2022,Winter,2022 Winter Olympics,Beijing


In [4]:
host_cities_df.drop(columns=['Unnamed: 0'], inplace=True)
host_cities_df

,Year,Season,Game,Host_City
0,1896,Summer,1896 Summer Olympics,Athina
1,1900,Summer,1900 Summer Olympics,Paris
2,1904,Summer,1904 Summer Olympics,St. Louis
3,1908,Summer,1908 Summer Olympics,London
4,1912,Summer,1912 Summer Olympics,Stockholm
...,...,...,...,...
57,2010,Winter,2010 Winter Olympics,Vancouver
58,2014,Winter,2014 Winter Olympics,Sochi
59,2018,Winter,2018 Winter Olympics,PyeongChang
60,2022,Winter,2022 Winter Olympics,Beijing


In [16]:
athletes_data_df

,id,name,gender,born,died,height,weight,noc,game,team,sport,event,position,medal
0,111427,Cui Peng,Male,31 May 1987,None,179 cm,72 kg,People's Republic of China,2008 Summer Olympics,CHN,Football (Football),"(Football (Football), Football, Men(Olympic),)","(13,)",
1,97418,Muzaffer Selçuk,Male,12 August 1961,None,180 cm,74 kg,Türkiye,1984 Winter Olympics,TUR,Cross Country Skiing (Skiing),"(Cross Country Skiing (Skiing), 15 kilometres,...","(DNF,)",
2,18579,Georges Breuille,Male,None,None,None,None,France,1920 Summer Olympics,FRA,Hockey,"(Hockey, Hockey, Men(Olympic),)","(4,)",
3,55113,Ángel Sposato,Male,17 May 1922,None,None,74 kg,Argentina,1952 Summer Olympics,ARG,Weightlifting,"(Weightlifting, Middleweight, Men(Olympic),)","(7,)",
4,143768,Udayan Mane,Male,24 February 1991,None,192 cm,None,India,2020 Summer Olympics,IND,Golf,"(Golf, Individual, Men(Olympic),)","(56,)",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321423,145001,Lee Hye-In,Female,16 January 1995,None,None,None,Republic of Korea,2020 Summer Olympics,KOR,Fencing,"(Fencing, Épée, Team, Women(Olympic),)","(2,)",Silver
321424,67556,Lauri Eskola,Male,16 November 1886,7 January 1958,None,None,Finland,1912 Summer Olympics,FIN,Athletics,"(Athletics, Cross-Country, Individual, Men(Oly...","(4,)",
321425,67556,Lauri Eskola,Male,16 November 1886,7 January 1958,None,None,Finland,1912 Summer Olympics,FIN,Athletics,"(Athletics, Cross-Country, Team, Men(Olympic),)","(2,)",Silver
321426,902192,Liu Chiu-Lung,Male,18 September 1956,None,None,None,Chinese Taipei,1984 Summer Olympics,TPE,Baseball (Baseball/Softball),"(Baseball (Baseball/Softball), Baseball, Men(O...","(3,)",


In [13]:
#athletes_data_df.drop(columns=['Unnamed: 0'], axis = 1, inplace = True)                             
athletes_data_df.rename(columns = {"noc" : "team", "team" : "noc"}, inplace = True)  
athletes_data_df.isnull().sum()

id               0
name             0
gender           0
born          5673
died        241559
height       81589
weight       87665
team             0
game             0
noc              0
sport            0
event            0
position         0
medal            0
dtype: int64

In [14]:
athletes_data_df.fillna("NULL", inplace = True)                                      
athletes_data_df.to_csv("data/athletes_data.csv", index = False)

In [15]:
athletes_data_df

,id,name,gender,born,died,height,weight,team,game,noc,sport,event,position,medal
0,111427,Cui Peng,Male,31 May 1987,NULL,179 cm,72 kg,People's Republic of China,2008 Summer Olympics,CHN,Football (Football),"(Football (Football), Football, Men(Olympic),)","(13,)",
1,97418,Muzaffer Selçuk,Male,12 August 1961,NULL,180 cm,74 kg,Türkiye,1984 Winter Olympics,TUR,Cross Country Skiing (Skiing),"(Cross Country Skiing (Skiing), 15 kilometres,...","(DNF,)",
2,18579,Georges Breuille,Male,NULL,NULL,NULL,NULL,France,1920 Summer Olympics,FRA,Hockey,"(Hockey, Hockey, Men(Olympic),)","(4,)",
3,55113,Ángel Sposato,Male,17 May 1922,NULL,NULL,74 kg,Argentina,1952 Summer Olympics,ARG,Weightlifting,"(Weightlifting, Middleweight, Men(Olympic),)","(7,)",
4,143768,Udayan Mane,Male,24 February 1991,NULL,192 cm,NULL,India,2020 Summer Olympics,IND,Golf,"(Golf, Individual, Men(Olympic),)","(56,)",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321423,145001,Lee Hye-In,Female,16 January 1995,NULL,NULL,NULL,Republic of Korea,2020 Summer Olympics,KOR,Fencing,"(Fencing, Épée, Team, Women(Olympic),)","(2,)",Silver
321424,67556,Lauri Eskola,Male,16 November 1886,7 January 1958,NULL,NULL,Finland,1912 Summer Olympics,FIN,Athletics,"(Athletics, Cross-Country, Individual, Men(Oly...","(4,)",
321425,67556,Lauri Eskola,Male,16 November 1886,7 January 1958,NULL,NULL,Finland,1912 Summer Olympics,FIN,Athletics,"(Athletics, Cross-Country, Team, Men(Olympic),)","(2,)",Silver
321426,902192,Liu Chiu-Lung,Male,18 September 1956,NULL,NULL,NULL,Chinese Taipei,1984 Summer Olympics,TPE,Baseball (Baseball/Softball),"(Baseball (Baseball/Softball), Baseball, Men(O...","(3,)",


In [33]:
athletes_data_df[athletes_data_df['id'] == 1]

,id,name,gender,born,died,height,weight,team,game,noc,sport,event,position,medal
126687,1,Jean-François Blanchy,Male,12 December 1886,2 October 1960,NULL,NULL,France,1912 Summer Olympics,FRA,Tennis,"(Tennis, Singles, Men(Olympic),)","(=17,)",
126688,1,Jean-François Blanchy,Male,12 December 1886,2 October 1960,NULL,NULL,France,1912 Summer Olympics,FRA,Tennis,"(Tennis, Doubles, Men(Olympic),)","(DNS,)",
126689,1,Jean-François Blanchy,Male,12 December 1886,2 October 1960,NULL,NULL,France,1920 Summer Olympics,FRA,Tennis,"(Tennis, Singles, Men(Olympic),)","(=32,)",
126690,1,Jean-François Blanchy,Male,12 December 1886,2 October 1960,NULL,NULL,France,1920 Summer Olympics,FRA,Tennis,"(Tennis, Doubles, Mixed(Olympic),)","(=8,)",
126691,1,Jean-François Blanchy,Male,12 December 1886,2 October 1960,NULL,NULL,France,1920 Summer Olympics,FRA,Tennis,"(Tennis, Doubles, Men(Olympic),)","(4,)",


In [34]:
athletes_data_df['id'].nunique()

155934

In [35]:
with gzip.open("data/athletes_content.json.gz", 'rb') as file:
    decompressed_data = file.read().decode('utf-8')

total_athletes_content = json.loads(decompressed_data)

In [36]:
def get_roles(athlete_data):
    
    athlete_info = BeautifulSoup(athlete_data, 'lxml')
    biography_table = athlete_info.find(attrs = {"class":"biodata"})
    
    if biography_table:
        biography = biography_table.find_all('tr')
        for row in biography:
            if row.find("th").text == 'Roles':   
                athlete_roles = {
                    'id' : int(athlete_info.find("h4").text.split("/")[-1]),
                    'name' : athlete_info.find("h1").text.strip(),
                    'roles' : row.find("td").text
                }
    return athlete_roles

In [37]:
start = perf_counter()
with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
    all_athletes_roles = list(executor.map(get_roles, total_athletes_content))   
end = perf_counter()
print(f"Elapsed Time: {end-start:.02f} seconds")

Elapsed Time: 1360.96 seconds


In [38]:
flattend_athletes_roles = [data for data in all_athletes_roles if data is not None]

In [39]:
athletes_roles_df = pd.DataFrame(flattend_athletes_roles)
athletes_roles_df.to_csv("data/athletes_roles.csv", index = False)
athletes_roles_df

,id,name,roles
0,111427,Cui Peng,Competed in Olympic Games
1,97418,Muzaffer Selçuk,Competed in Olympic Games
2,18579,Georges Breuille,Competed in Olympic Games
3,55113,Ángel Sposato,Competed in Olympic Games
4,143768,Udayan Mane,Competed in Olympic Games
...,...,...,...
155932,37771,Ralf Brudel,Competed in Olympic Games
155933,145001,Lee Hye-In,Competed in Olympic Games
155934,67556,Lauri Eskola,Competed in Olympic Games
155935,902192,Liu Chiu-Lung,Competed in Olympic Games (non-medal events)


In [2]:
athletes_data_df = pd.read_csv("data/athletes_data.csv")
athletes_data_df

,id,name,gender,born,died,height,weight,team,game,noc,sport,event,position,medal
0,111427,Cui Peng,Male,31 May 1987,NaN,179 cm,72 kg,People's Republic of China,2008 Summer Olympics,CHN,Football (Football),"('Football (Football), Football, Men(Olympic)',)","('13',)",NaN
1,97418,Muzaffer Selçuk,Male,12 August 1961,NaN,180 cm,74 kg,Türkiye,1984 Winter Olympics,TUR,Cross Country Skiing (Skiing),"('Cross Country Skiing (Skiing), 15 kilometres...","('DNF',)",NaN
2,18579,Georges Breuille,Male,NaN,NaN,NaN,NaN,France,1920 Summer Olympics,FRA,Hockey,"('Hockey, Hockey, Men(Olympic)',)","('4',)",NaN
3,55113,Ángel Sposato,Male,17 May 1922,NaN,NaN,74 kg,Argentina,1952 Summer Olympics,ARG,Weightlifting,"('Weightlifting, Middleweight, Men(Olympic)',)","('7',)",NaN
4,143768,Udayan Mane,Male,24 February 1991,NaN,192 cm,NaN,India,2020 Summer Olympics,IND,Golf,"('Golf, Individual, Men(Olympic)',)","('56',)",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321423,145001,Lee Hye-In,Female,16 January 1995,NaN,NaN,NaN,Republic of Korea,2020 Summer Olympics,KOR,Fencing,"('Fencing, Épée, Team, Women(Olympic)',)","('2',)",Silver
321424,67556,Lauri Eskola,Male,16 November 1886,7 January 1958,NaN,NaN,Finland,1912 Summer Olympics,FIN,Athletics,"('Athletics, Cross-Country, Individual, Men(Ol...","('4',)",NaN
321425,67556,Lauri Eskola,Male,16 November 1886,7 January 1958,NaN,NaN,Finland,1912 Summer Olympics,FIN,Athletics,"('Athletics, Cross-Country, Team, Men(Olympic)',)","('2',)",Silver
321426,902192,Liu Chiu-Lung,Male,18 September 1956,NaN,NaN,NaN,Chinese Taipei,1984 Summer Olympics,TPE,Baseball (Baseball/Softball),"('Baseball (Baseball/Softball), Baseball, Men(...","('3',)",NaN


In [16]:
athletes_data_df.isnull().sum()

id          0
name        0
gender      0
born        0
died        0
height      0
weight      0
team        0
game        0
noc         0
sport       0
event       0
position    0
medal       0
dtype: int64

In [19]:
athletes_data_df[athletes_data_df['id'] == 13748]

,id,name,gender,born,died,height,weight,team,game,noc,sport,event,position,medal
392,13748,Paul Brydon,Male,8 December 1951,NULL,173 cm,NULL,New Zealand,1972 Summer Olympics,NZL,Cycling Road (Cycling),"(Cycling Road (Cycling), Road Race, Individual...","(50,)",
393,13748,Paul Brydon,Male,8 December 1951,NULL,173 cm,NULL,New Zealand,1972 Summer Olympics,NZL,Cycling Track (Cycling),"(Cycling Track (Cycling), Team Pursuit, 4,000 ...","(14 r1/4,)",
